In [2]:
import pandas as pd
import numpy as np
from pandas import DataFrame as df

In [13]:
groups = {
    "A" : ['conll_2_in_gold.csv', 'conll_2_not_in_gold.csv'],
    "B" : ['conll_3_in_gold.csv', 'conll_3_not_in_gold.csv'],
    "C" : ['conll_3_train_in_gold.csv', 'conll_3_train_not_in_gold.csv'],
    "D" : ['conll_4_in_gold.csv', 'conll_4_not_in_gold.csv'],
    "E" : ['conll_4_train_in_gold.csv', 'conll_4_train_not_in_gold.csv']
}

In [102]:
sets = ['model_outputs/', 'human_labels/', 'human_labels_auditted/']

In [140]:
def combine_groups(group, current_set):
    """
    Takes in 2 files (_in_gold and _not_in_gold) from the same group and combines them
    Removes duplicates, and for human annotated files, filters out entries that had "None" as error type
    Returns a dataframe with 3 columns: num_models, span, entity_type
    """
    print("Combining files {} from folder {}".format(group, current_set))
    if current_set == 'model_outputs/':
        my_df = pd.read_csv(current_set+group[0], usecols=['num_models', 'corpus_span', 'corpus_ent_type'], encoding='latin-1')
        my_df_2 = pd.read_csv(current_set+group[1], usecols=['num_models', 'model_span', 'model_ent_type'], encoding='latin-1')
        # Adjust the numbers in `_not_in_gold.csv` files to match those in `_in_gold.csv` files
        my_df_2['num_models'] = 17 - my_df_2['num_models']
        my_df = my_df.rename(columns={"corpus_span": "span", "corpus_ent_type": "ent_type"})
        my_df_2 = my_df_2.rename(columns={"model_span": "span", "model_ent_type": "ent_type"})
        combined_df = pd.concat([my_df, my_df_2])
        combined_df = combined_df.drop_duplicates(subset=["span", "ent_type"])
    else:
        my_df = pd.read_csv(current_set+group[0], usecols=['num_models', 'corpus_span', 'corpus_ent_type', 'error_type'], encoding='latin-1')
        my_df_2 = pd.read_csv(current_set+group[1], usecols=['num_models', 'model_span', 'model_ent_type', 'error_type'], encoding='latin-1')
        # Adjust the numbers in `_not_in_gold.csv` files to match those in `_in_gold.csv` files
        my_df_2['num_models'] = 17 - my_df_2['num_models']
        my_df = my_df.rename(columns={"corpus_span": "span", "corpus_ent_type": "ent_type"})
        my_df_2 = my_df_2.rename(columns={"model_span": "span", "model_ent_type": "ent_type"})
        combined_df = pd.concat([my_df, my_df_2])
        combined_df = combined_df.drop_duplicates(subset=["span", "ent_type"])
        combined_df = combined_df.loc[combined_df['error_type'].isin(['Wrong','Token','Sentence','Span','Tag','Both'])]
        combined_df = combined_df.drop('error_type', axis=1)
    
    return combined_df

In [130]:
# Generate the 5 groups (A, B, C, D, E) for each of our 3 sets (model_outputs, human_annotator, human_auditor)
# final_set[group] is a list containing the 5 groups [A,B,C,D,E] for each set
final_set = {
    sets[0]: [],
    sets[1]: [],
    sets[2]: []
}
for s in sets:
    for g in groups:
        final_set[s].append(combine_groups(groups[g], s))

Combining files ['conll_2_in_gold.csv', 'conll_2_not_in_gold.csv'] from folder model_outputs/
Combining files ['conll_3_in_gold.csv', 'conll_3_not_in_gold.csv'] from folder model_outputs/
Combining files ['conll_3_train_in_gold.csv', 'conll_3_train_not_in_gold.csv'] from folder model_outputs/
Combining files ['conll_4_in_gold.csv', 'conll_4_not_in_gold.csv'] from folder model_outputs/
Combining files ['conll_4_train_in_gold.csv', 'conll_4_train_not_in_gold.csv'] from folder model_outputs/
Combining files ['conll_2_in_gold.csv', 'conll_2_not_in_gold.csv'] from folder human_labels/
Combining files ['conll_3_in_gold.csv', 'conll_3_not_in_gold.csv'] from folder human_labels/
Combining files ['conll_3_train_in_gold.csv', 'conll_3_train_not_in_gold.csv'] from folder human_labels/
Combining files ['conll_4_in_gold.csv', 'conll_4_not_in_gold.csv'] from folder human_labels/
Combining files ['conll_4_train_in_gold.csv', 'conll_4_train_not_in_gold.csv'] from folder human_labels/
Combining files [

In [161]:
def pairwise_compare(df1, df2):
    """
    This function compares pairwise the different dataframes we generated above
    We are interested in the number of overlapping (span, entity) pairs between the files
    Additionally we can also see how many models agreed with that pair (on average)
    Returns:
        - avg_overlap -> [0,1] indicates the % of entries overlapping between the 2 files on avg
        - mean_num_models -> the mean number of models which produced the outputs in the overlap
        - mean_num_models_diff -> the mean number of models by which the 2 files differ for the same row/record
    """
    df_joined = pd.merge(df1, df2, on=['span', 'ent_type'])
    # the "num_models_diff" field indicates what was the difference in the number of models which agreed to a certain (span, entity) pair
    df_joined['num_models_diff'] = abs(df_joined['num_models_x'] - df_joined['num_models_y'])
    avg_overlap = 0.5*((df_joined.shape[0]/df1.shape[0]) + (df_joined.shape[0]/df2.shape[0]))
    mean_num_models = 0.5*(df_joined['num_models_x'].mean() + df_joined['num_models_y'].mean())
    mean_num_models_diff = df_joined['num_models_diff'].mean()
    
    return avg_overlap, mean_num_models, mean_num_models_diff
    

In [162]:
pairwise_compare(final_set[sets[0]][0], final_set[sets[0]][1])

(0.6999704892137227, 13.899929912000623, 3.1595670119149597)

In [168]:
results = {
    sets[0] : [], 
    sets[1] : [],
    sets[2] : []
}

for s in final_set:
    results[s].append(pairwise_compare(final_set[s][0], final_set[s][1]))
    results[s].append(pairwise_compare(final_set[s][0], final_set[s][3]))
    results[s].append(pairwise_compare(final_set[s][1], final_set[s][3]))
    results[s].append(pairwise_compare(final_set[s][2], final_set[s][4]))

In [169]:
results

{'model_outputs/': [(0.6999704892137227,
   13.899929912000623,
   3.1595670119149597),
  (0.7030270777972292, 13.89784402241594, 3.178782689912827),
  (0.9023161429705182, 14.066916867299248, 0.747061322758816),
  (0.9310547283066118, 15.410332494890742, 0.5822590787612011)],
 'human_labels/': [(0.47276033289041186, 1.4213554987212276, 2.45985401459854),
  (0.43419509965622083, 1.5004064214590531, 2.267175572519084),
  (0.6857073471122406, 0.534855527638191, 0.4811083123425693),
  (0.8009103088810174, 0.6786992516539532, 0.5516372795969773)],
 'human_labels_auditted/': [(0.47323063450362235,
   1.426935737798769,
   2.4705882352941178),
  (0.43111715143578955, 1.4782776747062463, 2.246153846153846),
  (0.7184716389661088, 0.5469553963193282, 0.4847715736040609),
  (0.7948721802251311, 0.6578255083408127, 0.5329949238578681)]}